# 单步空间收敛测试

In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
from xfem import *
from xfem.lsetcurv import *
from math import pi,e
from numpy import linspace
import numpy as np

importing ngsxfem-2.1.2504


In [2]:
def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

In [3]:
def solve_biot_cutfem(dt, b, f, uD, pD, levelset, quad_mesh, mu,lam,tau_fpl,lambda_u,lambda_p,gamma_s,gamma_p,gamma_m,alpha,M,K, orderu, orderp, h=0.1):
    # 1. Construct the mesh
    square = SplineGeometry()
    square.AddRectangle((-1, -1), (1, 1), bc=1)
    ngmesh = square.GenerateMesh(maxh=h, quad_dominated=quad_mesh)
    mesh = Mesh(ngmesh)

    # 2. Get lset
    # Higher order level set approximation
    lsetmeshadap = LevelSetMeshAdaptation(mesh, order=1, threshold=0.1, discontinuous_qn=True)
    deformation = lsetmeshadap.CalcDeformation(levelset)
    lsetp1 = lsetmeshadap.lset_p1
    InterpolateToP1(levelset,lsetp1)

    # Cut information
    ci = CutInfo(mesh, lsetp1)
    hasneg = ci.GetElementsOfType(HASNEG)
    hasif = ci.GetElementsOfType(IF)
    ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)

    # 3. Define the unfitted fem space 
    Uhbase = VectorH1(mesh, order=orderu, dirichlet=[], dgjumps=True) # space for velocity
    Phbase = H1(mesh, order=orderp, dirichlet=[], dgjumps=True) # space for pressure
    U = Compress(Uhbase, GetDofsOfElements(Uhbase, ci.GetElementsOfType(HASNEG)))
    P = Compress(Phbase, GetDofsOfElements(Phbase, ci.GetElementsOfType(HASNEG)))
    fes = U*P
    (u,p), (v,q) = fes.TnT()
    gfu = GridFunction(fes)

    # Define special variables
    h = specialcf.mesh_size
    n = Normalize(grad(lsetp1))
    ne = specialcf.normal(2)
    strain_u = Sym(Grad(u))
    strain_v = Sym(Grad(v))

    # integration domains:
    dx = dCut(lsetp1, NEG, definedonelements=hasneg, deformation=deformation)
    ds = dCut(lsetp1, IF, definedonelements=hasif, deformation=deformation)
    dw = dFacetPatch(definedonelements=ba_facets, deformation=deformation)
    
    # 4. Define bilinear form
    ah = BilinearForm(fes)
    # Au
    ah += 2*mu*InnerProduct(strain_u,strain_v)*dx + lam*div(u)*div(v)*dx \
            - (InnerProduct(Stress(strain_u)*n,v) + InnerProduct(Stress(strain_v)*n,u) - lambda_u/h*InnerProduct(u,v))*ds
    # order=1 i_s 
    ah += gamma_s * h * InnerProduct(Grad(u)*ne - Grad(u.Other())*ne,Grad(v)*ne - Grad(v.Other())*ne) * dw
    # -B
    ah += -alpha*(div(v)*p*dx  - p*v*n*ds)
    # Ap
    ah += K*grad(p)*grad(q)*dx \
            - (K*grad(p)*n*q + K*grad(q)*n*p - lambda_p/h*p*q)*ds
    # order=1 i_p 
    ah += gamma_p * h * (grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dw
    # FPL stablization
    ah += tau_fpl*grad(p)*grad(q)*dx

    ah.Assemble()
    
    # r.h.s
    lh = LinearForm(fes)
    lh += b*v*dx - InnerProduct(uD,Stress(Sym(Grad(v)))*n)*ds + lambda_u/h*uD*v*ds
    lh += f*q*dx - K*grad(q)*n*pD*ds + lambda_p/h*pD*q*ds
    lh.Assemble()

    
    gfu.vec.data = ah.mat.Inverse() * lh.vec
    error_u = sqrt(Integrate((gfu.components[0] - exact_u)**2 * dx, mesh))
    error_p = sqrt(Integrate((gfu.components[1] - exact_p)**2 * dx, mesh))
    

    return error_u,error_p, gfu.space.ndof,


In [4]:
def print_convergence_table(results):
    print(f"{'h':>8} | {'DoFs':>8} | {'u_L2 Error':>12} | {'Order':>6} | {'p_L2 Error':>12} | {'Order':>6}")
    print("-" * 60)
    for i, (h0, dofs, erroru,errorp) in enumerate(results):
        if i == 0:
            print(f"{h0:8.4f} | {dofs:8d} | {erroru:12.4e} | {'-':>6}| {errorp:12.4e} | {'-':>6}")
        else:
            prev_h, _, prev_erroru,prev_errorp = results[i-1]
            rate_u = (np.log(prev_erroru) - np.log(erroru)) / (np.log(prev_h) - np.log(h0))
            rate_p = (np.log(prev_errorp) - np.log(errorp)) / (np.log(prev_h) - np.log(h0))
            print(f"{h0:8.4f} | {dofs:8d} | {erroru:12.4e} | {rate_u:6.2f}| {errorp:12.4e} | {rate_p:6.2f}")


In [5]:
# Define physical parameters for Biot
E = 1
nu = 0.2
mu  = E/2/(1+nu)
lam = E*nu/(1+nu)/(1-2*nu)
K = 0.1
alpha = 1
M = 100

# Quadrilateral (or simplicial mesh)
quad_mesh = True
# Finite element space order
orderu = 1
orderp = 1
# time step
# dt = 1e-4

# penalty parameters
# lambda_u = 20*lam
# lambda_p = 50*K
# gamma_s = 20*lam
# gamma_p = 10*K

lambda_u = 100
lambda_p = 500*K
gamma_s = 20*lam
gamma_p = 10*K
# gamma_m = 0.1/M/dt
gamma_m = 0 


# Manufactured exact solution for monitoring the error
t = Parameter(0.0) # A Parameter is a constant CoefficientFunction the value of which can be changed with the Set-function.
u_x = e**(-t)*sin(pi*x)*sin(pi*y)
u_y = e**(-t)*sin(pi*x)*sin(pi*y)
exact_u = CF((u_x,u_y))
exact_p = e**(-t)*(cos(pi*y)+1)

# strain tensor
epsilon_xx = u_x.Diff(x)
epsilon_yy = u_y.Diff(y) 
epsilon_xy = 0.5*(u_x.Diff(y) +  u_y.Diff(x))

# total stress tensor
sigma_xx = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_xx - alpha*exact_p
sigma_yy = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_yy - alpha*exact_p
sigma_xy = 2*mu*epsilon_xy

# 右端项 f_x, f_y
f_x = - (sigma_xx.Diff(x) + sigma_xy.Diff(y))
f_y = - (sigma_xy.Diff(x) + sigma_yy.Diff(y))

# 向量形式
b = CF( (f_x,f_y) ) # body force 
f = (1/M*exact_p+alpha*(u_x.Diff(x)+u_y.Diff(y))).Diff(t) - K*(exact_p.Diff(x).Diff(x)+exact_p.Diff(y).Diff(y)) # source term

# uD = exact_u
# pD = exact_p

# Define level set function
levelset = sqrt(x**2 + y**2) - 0.5


In [6]:
results = []

for k in range(2, 6):
    h0 = 1/2**k
    # tau_fpl = 0.1*h0**2
    tau_fpl = 0
    error_u,error_p, ndof, = solve_biot_cutfem(dt, b, f, exact_u, exact_p, levelset, quad_mesh, \
                                            mu,lam,tau_fpl,lambda_u,lambda_p,gamma_s,gamma_p,gamma_m,alpha,M,K, orderu, orderp, h0)
    results.append((h0,ndof,error_u, error_p ))

print_convergence_table(results)


       h |     DoFs |   u_L2 Error |  Order |   p_L2 Error |  Order
------------------------------------------------------------
  0.2500 |       75 |   4.2456e-02 |      -|   1.7246e-01 |      -
  0.1250 |      231 |   4.5402e-02 |  -0.10|   2.6081e-01 |  -0.60
  0.0625 |      771 |   4.9007e-02 |  -0.11|   2.8632e-01 |  -0.13
  0.0312 |     2763 |   5.0007e-02 |  -0.03|   2.9214e-01 |  -0.03
